In [ ]:
!pip install qdrant-client[fastembed]

In [ ]:
from qdrant_client import QdrantClient,models

In [ ]:
from google.colab import userdata

In [ ]:
qdrant_client = QdrantClient(
    url="https://98660ff7-80b0-4946-99a4-45883da637e1.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key=userdata.get('qdrant'),
)

In [ ]:
model_name = "sentence-transformers/all-MiniLM-L6-v2" # embedding - 384

In [ ]:
payload = [
    {"document": "Mohan goes to school", "source": "Langchain-docs", },
    {"document": "Ramesh is a banker", "source": "LlamaIndex-docs"},
    {"document": "Anita is working on a project", "source": "Langchain-docs", },
    {"document": "Kriti has some books", "source": "LlamaIndex-docs"},
]

In [ ]:
docs = [models.Document(text=data["document"], model=model_name) for data in payload]
ids = [1, 2, 3, 4]

In [ ]:
docs

[Document(text='Mohan goes to school', model='sentence-transformers/all-MiniLM-L6-v2', options=None),
 Document(text='Ramesh is a banker', model='sentence-transformers/all-MiniLM-L6-v2', options=None),
 Document(text='Anita is working on a project', model='sentence-transformers/all-MiniLM-L6-v2', options=None),
 Document(text='Kriti has some books', model='sentence-transformers/all-MiniLM-L6-v2', options=None)]

In [ ]:
qdrant_client.create_collection(
    "demo_collection",
    vectors_config=models.VectorParams(
        size=qdrant_client.get_embedding_size(model_name), distance=models.Distance.COSINE)
)

True

In [ ]:
qdrant_client.upload_collection(
    collection_name="demo_collection",
    vectors=docs,
    ids=ids,
    payload=payload,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
search_result = qdrant_client.query_points(
    collection_name="demo_collection",
    query=models.Document(text="Who has books", model=model_name)
).points


In [ ]:
for result in search_result:
  print(result)

id=4 version=1 score=0.5690124 payload={'document': 'Kriti has some books', 'source': 'LlamaIndex-docs'} vector=None shard_key=None order_value=None
id=3 version=1 score=0.16900815 payload={'document': 'Anita is working on a project', 'source': 'Langchain-docs'} vector=None shard_key=None order_value=None
id=2 version=1 score=0.16262642 payload={'document': 'Ramesh is a banker', 'source': 'LlamaIndex-docs'} vector=None shard_key=None order_value=None
id=1 version=1 score=0.12911835 payload={'document': 'Mohan goes to school', 'source': 'Langchain-docs'} vector=None shard_key=None order_value=None


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
from groq import Groq

In [ ]:
groq_client = Groq(
    # This is the default and can be omitted
    api_key=userdata.get("GROQ_KEY"),
)


In [ ]:
def get_completion(prompt):
  completion = groq_client.chat.completions.create(
      model="openai/gpt-oss-120b",
      messages=[
        {
          "role": "user",
          "content": prompt
        }
      ],
      temperature=1,
      # reasoning_effort="medium",
      stream=True,
  )

  return completion


In [ ]:
query = "Who has books"

In [ ]:
search_result = qdrant_client.query_points(
    collection_name="demo_collection",
    query=models.Document(text=query, model=model_name)
).points


In [ ]:
for result in search_result[:2]:
  print(result.payload["document"])

Kriti has some books
Anita is working on a project


In [ ]:
system_prompt = "You're a helpful AI assistant. You will only answer based on information available to you in the DOCUMENTS section, no other source or internal knowledge has to be used. Section start: DOCUMENTS "

for result in search_result[:2]:
  system_prompt += result.payload["document"] + "\n"

system_prompt += " Section end: DOCUMENT.    User's query: "


In [ ]:
system_prompt

"You're a helpful AI assistant. You will only answer based on information available to you in the DOCUMENTS section, no other source or internal knowledge has to be used. Section start: DOCUMENTS Kriti has some books\nAnita is working on a project\n Section end: DOCUMENT.    User's query: "

In [ ]:
for chunk in get_completion(system_prompt + query):
    print(chunk.choices[0].delta.content or "", end="")

Kriti has some books.